In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [2]:
# CHANGE ME!
TASK_ID = 'changing_mean'
AUTHOR = 'g'


In [16]:
task = Task(
    id=TASK_ID,
    name="Changing mean",
    description="Return index of the first element you can remove that does not change the mean of the list by more than 0.1",
    plan=[
        {
        "id": "remove",
        "description": "remove",
    }, {
        "id": "orig",
        "description": "mean of the list",
    }, {
        "id": "new",
        "description": "change the mean",
    }, 
        {
        "id": "diff",
        "description": "by more than 0.1",
    }, 
    ],
    sample_input={
       "vals":[{"id":0,"value":1.0},{"id":1,"value":2.0},{"id":2,"value":3.0},{"id":3,"value":1.0}],
    },
    sample_output=1,
    category="Basic",
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [4]:
imp = replace(prototype,
    language='python-imperative',
    source='''
def get_mean(ls):
  sum = 0
  for e in ls:
    sum += e['value']
  return sum/len(ls)

def changing_mean(values):
  start_mean = get_mean(values)
  for i,elem in enumerate(values):
    new_ls = values[:i] + values[i+1:]
    mean = get_mean(new_ls)
    if abs(mean - start_mean) < 0.1:
      return i
  return None
''')
imp.execute(task)
imp.save()

In [5]:
func = replace(prototype,
    language='python-functional',
    source='''
def changing_mean(values):
  start_mean = sum([l['value'] for l in values])/len(values)
  diff = lambda m, sm: abs(m - sm)
    
  for i,elem in enumerate(values):
    new_ls = [x['value'] for x in values if x != elem]
    mean = sum(new_ls)/len(new_ls)
    if diff(mean,start_mean) < 0.1:
      return i
  return None
''')
func.execute(task)
func.save()

In [34]:
pandas = replace(prototype,
    language='python-pandas',
    source='''
def changing_mean(vals):
  mean = vals.value.mean()
  mean_without = vals.apply(lambda row: vals[vals.id != row.id].value.mean(), axis=1)
  return vals[(mean_without - mean).abs() < 0.1].id.tolist()
''')
pandas.execute(task)
pandas.save()

In [12]:
#NOTE: Issue with float conversion
func = replace(prototype,
    language='datalog',
    source='''.decl avg_except(I:number, Avg:float)    
avg_except(I, Avg) :- 
  values(I, _),
  N = sum 1.0 : { values(J, _), I != J },
  Total = sum V : { values(J, V), I != J },
  Avg = Total / N.
  
changing_mean(I) :-
  values(I, _),
  N = sum 1.0 : values(_, _),
  Total = sum V : values(_, V),
  Avg = Total / N,
  avg_except(I, AvgExcept),
  ((Avg > AvgExcept, Avg - AvgExcept < 0.1);
   (Avg < AvgExcept, AvgExcept - Avg < 0.1)).''')
func.execute(task)
func.save()

In [18]:
#NOTE issue with getting average of dataset without current value
func = replace(prototype,
    language='sql',
    source='''SELECT id
FROM vals v1
WHERE ABS((
  SELECT AVG(value)
  FROM vals v2
  WHERE v1.id != v2.id
) - (SELECT AVG(value) FROM vals)) < 0.1''')
func.execute(task)
func.save()